In [6]:
from logging import error, exception
# from httplib2.error import FailedToDecompressContent
import pandas as pd
import pygsheets
import os
from datetime import datetime, date
import sys

import gsheet_utility as gsheetUtility
import Email_Code as fkEmail
import data_processing as data_processing
import numpy as np

basepath = '/Users/a/Documents/GitHub/exception_visibility/'

In [4]:
basePath = '/Users/a/Documents/GitHub/exception_visibility/'
# basepath = '/Users/a/Documents/GitHub/exception_visibility/'
date_master_file = pd.read_csv( '/Users/a/Documents/GitHub/exception_visibility/Dashboard/data/week_details.csv')
date_master_file['weekday'] = pd.to_datetime(date_master_file["weekday"],format = '%d/%m/%Y').dt.date
date_master_file['month_year'] = pd.to_datetime(date_master_file["month_year"],format = '%d/%m/%Y').dt.date

rc_raw_data_location = basePath + 'ETL/RC_Raw_Data/'
mh_raw_data_location = basePath + 'ETL/MH_Raw_Data/'
orphan_raw_data_location = basePath + 'ETL/Orphan_Raw_Data/'
hv_orphan_raw_data_location = basePath + 'ETL/HV_Orphan_Raw_Data/'

today = date.today()
today = str(today.year) + "-" + str('{:02d}'.format(today.month)) + "-" + str('{:02d}'.format(today.day))

rcgsheetdatapath = basePath + 'ETL/gsheet_data/rc_data/' + today + '.csv'
mhgsheetdatapath = basePath + 'ETL/gsheet_data/mh_data/' + today + '.csv'
orphangsheetdatapath = basePath + 'ETL/gsheet_data/orphan_data/' + today + '.csv'
hv_orphangsheetdatapath = basePath + 'ETL/gsheet_data/hv_orphan_data/' + today + '.csv'

hub_zone_data = gsheetUtility.get_gsheet_data('https://docs.google.com/spreadsheets/d/187u3lIk3GSDiHuUm-ZLno_keTe-jCAu1bqiTon7ExgU/edit?usp=sharing', 'A3', orphangsheetdatapath,'Hubdetails')

error_message = ''
html_message = ''

01/07/2022, 11:32:32 Starting Fetching Gsheet Data
01/07/2022, 11:32:34 Fetching Gsheet Completed
Hubdetails
A3


In [16]:
html_message = html_message + "Data Ingestion Job for Orphan form started at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting MH data process')
html_message = html_message + "Starting to fetch orphan raw data from google sheet : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'

print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting orphan data process')
orphanData = gsheetUtility.get_gsheet_data('https://docs.google.com/spreadsheets/d/1QrfuwxkbDHSrMDREnhicUcA2dPqpZgJqvFQ7f9FHRvU/edit?usp=sharing', 'A3', orphangsheetdatapath,'Orphan')
orphandbcolumns = ['shipment_value', 'weeknum', 'cleared_shipment_tracking_id', 'month', 'date', 'scanned_timestamp', 'motherhub_name',	'shipment_category', 'orphan_reason', 
                    'is_invoice_available', 'shipment_type', 'content_details', 'lane_details_semi_large', 'consignment_id_semi_large', 'bag_id',	'orphan_idnetified_mh_area', 'image_url', 
                    'orphan_id', 'bag_seal_id', 'seller_name', 'seller_id', 'seller_type']
# orphanGsheetColumns = orphanData.columns
html_message = html_message + "completed fetching orphan raw data from google sheet : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Fetching orphan data process Completed')
orphanData = gsheetUtility.assignDBColumns(orphanData, gsheet_asset='MH Orphan Data', dbColumns=orphandbcolumns)
orphanData = orphanData.drop(columns=['weeknum','month','date'])
orphanData = data_processing.datatype_conversion(orphanData, 'scanned_timestamp', 'datetime')
orphanData['scanned_date'] = orphanData['scanned_timestamp'].dt.date
orphanData = data_processing.fetch_date_details(orphanData, 'exception_log_timestamp', date_master_file)

orphanData.cleared_shipment_tracking_id = orphanData.cleared_shipment_tracking_id.replace(r'^\s*$', np.NaN, regex=True)
orphanData['is_tracking_id_available'] = ''
orphanData.loc[orphanData.cleared_shipment_tracking_id.isna(), ['is_tracking_id_available']]='No'
orphanData.loc[~orphanData.cleared_shipment_tracking_id.isna(), ['is_tracking_id_available']] = 'Yes'
# print(df['is_tracking_id_available'])
# orphanData['age']=0
# orphanData['age'] = (datetime.today().date() - pd.to_datetime(orphanData['scanned_date']).dt.date)/np.timedelta64(1, 'D')
# # print(orphanData['age'])
# orphanData['ageing_category'] = orphanData['age'].apply(lambda x: '1st week' if x <= 7 else ('2nd week' if ((x >7) & (x <= 14) ) else ('3rd week') if ((x >14) & (x <= 21) ) else ('4th week' if ((x >21) & (x <= 27) ) else 'Older than 4 weeks')))
# print(orphanData['ageing_category'])

# pd.merge(orphanData, hub_zone_data, left_on='motherhub_name', right_on='hub_name', how='left')
orphanData.motherhub_name = orphanData.motherhub_name.str.upper()
orphanData = pd.merge(orphanData, hub_zone_data, left_on='motherhub_name', right_on='hub_name', how='left')

print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting to create orphan raw data files')
html_message = html_message + "Starting to create orphan raw data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
orphanData['shipment_value'] = orphanData['shipment_value'].str.replace(',', '')
orphanData['shipment_value'] = orphanData['shipment_value'].str.replace('.0', '')
orphanData.cleared_shipment_tracking_id = orphanData.cleared_shipment_tracking_id.str.replace(r'\r', '')
orphanData.shipment_value = orphanData.shipment_value.str.replace(' ', '')
orphanData.shipment_value = orphanData.shipment_value.fillna(0)
orphanData.shipment_value =pd.to_numeric(orphanData.shipment_value)
orphanData.orphan_id = orphanData.orphan_id.str.upper()
filenames = []
created_raw_files = data_processing.fetch_created_files(orphan_raw_data_location)
filenames = data_processing.create_rewrite_raw_files(orphanData,'scanned_date', created_raw_files, orphan_raw_data_location )
html_message = html_message + "Completed creating orphan raw data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Completed creating orphan raw data files')



12/27/2021 12:50:01 Starting MH data process
12/27/2021 12:50:01 Starting orphan data process
12/27/2021, 12:50:01 Starting Fetching Gsheet Data
12/27/2021, 12:50:11 Fetching Gsheet Completed
Orphan
A3
12/27/2021 12:50:12 Fetching orphan data process Completed
Converting scanned_timestamp to datetime
Converted scanned_timestamp to datetime
mapping dates and weekend to scanned date
12/27/2021 12:50:29 Starting to create orphan raw data files
fetching names of already created raw files
started creating raw files
creating raw file for the date 2021-08-01.csv
creating raw file for the date 2021-08-02.csv
creating raw file for the date 2021-08-03.csv
creating raw file for the date 2021-08-04.csv
creating raw file for the date 2021-08-05.csv
creating raw file for the date 2021-08-06.csv
creating raw file for the date 2021-08-07.csv
creating raw file for the date 2021-08-08.csv
creating raw file for the date 2021-08-09.csv
creating raw file for the date 2021-08-10.csv
creating raw file for th

In [43]:
print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting to collate data for dashboard')
html_message = html_message + "Starting to create orphan raw data files at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
orphan_final_columns = ['shipment_value', 'cleared_shipment_tracking_id', 'scanned_timestamp', 'motherhub_name',	'shipment_category', 'orphan_reason', 
                        'is_invoice_available', 'shipment_type', 'orphan_idnetified_mh_area', 'image_url', 
                        'orphan_id', 'scanned_date', 'weekend', 'month', 'year', 'month_year','weeknum','is_tracking_id_available','asset']
created_raw_files = data_processing.fetch_created_files(orphan_raw_data_location)
orphan_dashboard_data = data_processing.collate_data_for_dashboard(datetime.now().date(), 180, created_raw_files, orphan_raw_data_location, orphan_final_columns)
orphan_dashboard_data.to_csv(basePath + 'Dashboard/data/orphan_full_data.csv', index=False)

12/22/2021 13:30:57 Starting to collate data for dashboard
fetching names of already created raw files
2021-11-30.csv
2021-11-29.csv
2021-11-28.csv
2021-11-27.csv
2021-11-26.csv
2021-11-25.csv
2021-11-24.csv
2021-11-23.csv
2021-11-22.csv
2021-11-21.csv
2021-11-20.csv
2021-11-19.csv
2021-11-18.csv
2021-11-17.csv
2021-11-16.csv
2021-11-15.csv
2021-11-14.csv
2021-11-13.csv
2021-11-12.csv
2021-11-11.csv
2021-11-10.csv
2021-11-09.csv
2021-11-08.csv
2021-11-07.csv
2021-11-06.csv
2021-11-05.csv
2021-11-04.csv
2021-11-03.csv
2021-11-02.csv
2021-11-01.csv
2021-10-31.csv
2021-10-30.csv
2021-10-29.csv
2021-10-28.csv
2021-10-27.csv
2021-10-26.csv
2021-10-25.csv
2021-10-24.csv
2021-10-23.csv
2021-10-22.csv
2021-10-21.csv
2021-10-20.csv
2021-10-19.csv
2021-10-18.csv
2021-10-17.csv
2021-10-16.csv
2021-10-15.csv
2021-10-14.csv
2021-10-13.csv
2021-10-12.csv
2021-10-11.csv
2021-10-10.csv
2021-10-09.csv
2021-10-08.csv
2021-10-07.csv
2021-10-06.csv
2021-10-05.csv
2021-10-04.csv
2021-10-03.csv
2021-10-02.c

In [17]:
orphanData.columns

Index(['shipment_value', 'cleared_shipment_tracking_id', 'scanned_timestamp',
       'motherhub_name', 'shipment_category', 'orphan_reason',
       'is_invoice_available', 'shipment_type', 'content_details',
       'lane_details_semi_large', 'consignment_id_semi_large', 'bag_id',
       'orphan_idnetified_mh_area', 'image_url', 'orphan_id', 'bag_seal_id',
       'seller_name', 'seller_id', 'seller_type', 'asset', 'scanned_date',
       'weekend', 'month', 'year', 'month_year', 'weeknum',
       'is_tracking_id_available', 'Asset_Name', 'zone', 'hub_name'],
      dtype='object')

In [15]:
orphanData[orphanData.orphan_id=='COS03795963-20']

117483    Orphan Data
Name: asset, dtype: object

In [15]:
pd.DataFrame(pd.pivot_table(orpdf, index=['orphan_idnetified_mh_area'], columns=date_agg, fill_value=0, values=['scanned_timestamp'], aggfunc=len)).reset_index().rename(columns={'scanned_timestamp': 'orphan_count'})

0                               Orphan (Non-damage)
1                                   Prone to Orphan
10                                  Orphan (Damage)
54                        Orphan (Partial shipment)
13635    Orphan from FC (Without packaging / label)
Name: shipment_category, dtype: object

In [27]:
'FMPC1526106994\r'.replace('\r','')

'FMPC1526106994'

In [12]:
orphanData.motherhub_name = orphanData.motherhub_name.str.upper()
pd.merge(orphanData, hub_zone_data, left_on='motherhub_name', right_on='hub_name', how='left')

,shipment_value,cleared_shipment_tracking_id,scanned_timestamp,motherhub_name,shipment_category,orphan_reason,is_invoice_available,shipment_type,content_details,lane_details_semi_large,...,scanned_date,weekend,month,year,month_year,weeknum,is_tracking_id_available,Asset_Name,zone,hub_name
0,438.0,FMPP0882155736,2021-08-01 00:00:42,MOTHERHUB_OFX,Orphan (Non-damage),Label peeled off,No,Market Place shipment,HOT WATER BAG,NA,...,2021-08-01,07/08/2021,August,2021,2021-08-01,32,Yes,MotherHub_OFX,North,MOTHERHUB_OFX
1,NaN,FMPC1328102002,2021-08-01 00:01:37,MOTHERHUB_RAI,Prone to Orphan,Damage packaging,No,FC Shipment,pahadi lala Beautiful Handicraft Wooden 3-Tier...,MotherHub_BNO-RAI,...,2021-08-01,07/08/2021,August,2021,2021-08-01,32,Yes,MotherHub_RAI,East,MOTHERHUB_RAI
2,NaN,NaN,2021-08-01 00:02:06,MOTHERHUB_OFX,Orphan (Non-damage),Label peeled off,No,Market Place shipment,HOT WATER BAG,NA,...,2021-08-01,07/08/2021,August,2021,2021-08-01,32,No,MotherHub_OFX,North,MOTHERHUB_OFX
3,NaN,NaN,2021-08-01 00:03:57,MOTHERHUB_OFX,Orphan (Non-damage),Label peeled off,No,Market Place shipment,HOT WATER BAG,NA,...,2021-08-01,07/08/2021,August,2021,2021-08-01,32,No,MotherHub_OFX,North,MOTHERHUB_OFX
4,NaN,NaN,2021-08-01 00:05:11,MOTHERHUB_OFX,Orphan (Non-damage),Label peeled off,No,Market Place shipment,HOT WATER BAG,NA,...,2021-08-01,07/08/2021,August,2021,2021-08-01,32,No,MotherHub_OFX,North,MOTHERHUB_OFX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117722,NaN,NaN,2021-11-30 23:54:06,MOTHERHUB_JKS_RET,Orphan (Non-damage),Label peeled off,No,FC Shipment,intex fan blade,NA,...,2021-11-30,04/12/2021,November,2021,2021-11-01,49,No,MotherHub_JKS_Ret,North,MOTHERHUB_JKS_RET
117723,NaN,NaN,2021-11-30 23:56:05,MOTHERHUB_JKS_RET,Orphan (Non-damage),Label peeled off,No,FC Shipment,sefty shoes,NA,...,2021-11-30,04/12/2021,November,2021,2021-11-01,49,No,MotherHub_JKS_Ret,North,MOTHERHUB_JKS_RET
117724,NaN,NaN,2021-11-30 23:57:21,MOTHERHUB_GGN,Orphan (Non-damage),NPG shipments,No,FC Shipment,NIVIA BAG,NA,...,2021-11-30,04/12/2021,November,2021,2021-11-01,49,No,MotherHub_GGN,North,MOTHERHUB_GGN
117725,NaN,NaN,2021-11-30 23:57:29,MOTHERHUB_JKS_RET,Orphan (Non-damage),Label peeled off,No,FC Shipment,book,NA,...,2021-11-30,04/12/2021,November,2021,2021-11-01,49,No,MotherHub_JKS_Ret,North,MOTHERHUB_JKS_RET


In [7]:
orphan_df = pd.read_csv(basepath +'/Dashboard/data/orphan_full_data.csv', low_memory=False)

In [13]:
orphan_df['weeknum_year'] = orphan_df.weeknum.astype(str) + '-' + orphan_df.year.astype(str)

In [9]:
orphan_df.asset

0         Orphan Data
1         Orphan Data
2         Orphan Data
3         Orphan Data
4         Orphan Data
             ...     
169228    Orphan Data
169229    Orphan Data
169230    Orphan Data
169231    Orphan Data
169232    Orphan Data
Name: asset, Length: 169233, dtype: object

In [27]:
pd.to_datetime('2021-12-28T12:57:53.026248').date()

datetime.date(2021, 12, 28)

In [29]:
start_date = pd.to_datetime('2021-12-28T12:57:53.026248').date()
end_date = pd.to_datetime('2021-12-28T12:57:53.026248').date()
(end_date - start_date).days

0